### Panoramica dei test privacy-preserving

Questa sezione estende il notebook con una guida organica ai 5 scenari A–E, una sintesi teorica di Differential Privacy (DP) e Homomorphic Encryption (HE), e codice eseguibile per caricare e verificare i risultati di ciascuno scenario.

- **Scopo**: rendere riproducibili e verificabili i test, con spiegazioni e controlli rapidi.
- **Cosa trovi qui**:
  - **Setup rapido** per import e percorsi.
  - **Relazioni teoriche** tra HE e DP e il loro impatto atteso.
  - **Caricamento summary esteso** con metriche aggiuntive (rounds, HE banda, DP, ecc.).
  - **Utility per scenari A–E**: contesto, assunti, risultati attesi e funzione di check.
  - **Codice per eseguire i controlli** per ogni scenario in modo indipendente.

Nota: non rimuoviamo sezioni esistenti; aggiungiamo solo contenuti integrativi e non distruttivi.


In [ ]:
# Setup rapido: import, percorsi, utilità
from pathlib import Path
import json
import pandas as pd
import numpy as np

BASE = Path("experiments_privacy")
RES_DIR = BASE / "results"
SUMMARY_JSON = RES_DIR / "summary.json"
SUMMARY_CSV = RES_DIR / "summary.csv"

# Caricamento summary esteso (generato da experiments_privacy/shared/summary.py)
def load_extended_summary():
    if SUMMARY_JSON.exists():
        with open(SUMMARY_JSON, "r") as f:
            data = json.load(f)
        return pd.DataFrame(data)
    elif SUMMARY_CSV.exists():
        return pd.read_csv(SUMMARY_CSV)
    else:
        raise FileNotFoundError("Nessun summary trovato. Esegui summarize_and_save prima.")

extended_summary_df = None
try:
    extended_summary_df = load_extended_summary()
    display(extended_summary_df)
except Exception as e:
    print("[WARN] Impossibile caricare summary esteso:", e)



### Relazioni teoriche: Differential Privacy (DP) e Homomorphic Encryption (HE)

- **DP**: garantisce privacy aggiungendo rumore controllato. Nella pratica (DP-SGD o varianti) introduce un trade-off tra **accuratezza** e **privacy**; la privacy è parametrizzata da \(\epsilon\) (più basso = maggiore privacy) e \(\delta\). Atteso: un calo di accuracy all'aumentare del rumore, con tempi simili di training locale, ma potenziale necessità di più round.
- **HE**: cifratura omomorfica per eseguire operazioni su dati cifrati. Qui stimiamo l'overhead di comunicazione (dimensione ciphertext). Atteso: **accuracy invariata** (se si cifrano solo pesi/aggiornamenti), **overhead in banda** e possibili costi computazionali.
- **DP + HE**: composizione in cui DP protegge i gradienti/pesi e HE protegge il **transito** e l'aggregazione. Atteso: combinazione degli effetti, con accuracy calante per DP e banda crescente per HE.

Dove si riflette nel nostro codice:
- `use_dp` controlla rumore/clipping su pesi locali.
- `use_he` abilita la stima `he_round_overheads` con dimensioni ciphertext per round.
- Nel summary esteso aggiungiamo colonne: `Epsilon`, `Delta`, `UseDP`, `UseHE`, `HE_MB_per_round`, `HE_MB_total`, `HE_MB_per_sec` e statistiche dei tempi per round.


In [ ]:
# Utility: loader e checker per scenari A–E
from typing import Optional, Dict, Any

def load_scenario_row(df: pd.DataFrame, scenario: str) -> Optional[pd.Series]:
    if df is None:
        return None
    rows = df[df["Scenario"] == scenario]
    if len(rows) == 0:
        return None
    return rows.iloc[0]


def check_scenario(df: pd.DataFrame, scenario: str, expectations: Dict[str, Any]) -> Dict[str, Any]:
    """Verifica regole base per uno scenario.
    expectations: dict con chiavi opzionali tra:
      - expect_use_dp (bool)
      - expect_use_he (bool)
      - min_accuracy (float) / max_accuracy (float)
      - max_he_mb_per_round (float)
      - expect_epsilon_not_null (bool)
    """
    res = {"scenario": scenario, "passed": True, "violations": []}
    row = load_scenario_row(df, scenario)
    if row is None:
        res["passed"] = False
        res["violations"].append("scenario non trovato nel summary")
        return res
    # Regole
    if "expect_use_dp" in expectations and not pd.isna(row.get("UseDP")):
        if bool(row.get("UseDP")) != bool(expectations["expect_use_dp"]):
            res["passed"] = False
            res["violations"].append(f"UseDP atteso {expectations['expect_use_dp']}, trovato {row.get('UseDP')}")
    if "expect_use_he" in expectations and not pd.isna(row.get("UseHE")):
        if bool(row.get("UseHE")) != bool(expectations["expect_use_he"]):
            res["passed"] = False
            res["violations"].append(f"UseHE atteso {expectations['expect_use_he']}, trovato {row.get('UseHE')}")
    if "min_accuracy" in expectations and not pd.isna(row.get("Accuracy")):
        if float(row.get("Accuracy")) < float(expectations["min_accuracy"]):
            res["passed"] = False
            res["violations"].append(f"Accuracy {row.get('Accuracy'):.3f} < min {expectations['min_accuracy']}")
    if "max_accuracy" in expectations and not pd.isna(row.get("Accuracy")):
        if float(row.get("Accuracy")) > float(expectations["max_accuracy"]):
            res["passed"] = False
            res["violations"].append(f"Accuracy {row.get('Accuracy'):.3f} > max {expectations['max_accuracy']}")
    if "max_he_mb_per_round" in expectations and not pd.isna(row.get("HE_MB_per_round")):
        if float(row.get("HE_MB_per_round")) > float(expectations["max_he_mb_per_round"]):
            res["passed"] = False
            res["violations"].append(f"HE_MB_per_round {row.get('HE_MB_per_round'):.3f} > max {expectations['max_he_mb_per_round']}")
    if expectations.get("expect_epsilon_not_null", False):
        if pd.isna(row.get("Epsilon")):
            res["passed"] = False
            res["violations"].append("Epsilon atteso non nullo, trovato NULL")
    res["row"] = row
    return res

# Esempio veloce (non fallisce se summary manca)
if extended_summary_df is not None:
    print(check_scenario(extended_summary_df, "A", {"expect_use_dp": False, "expect_use_he": False}))



### Scenari A–E: contesto, assunti, risultati attesi

Di seguito formalizziamo i 5 scenari e aggiungiamo per ciascuno un controllo automatico. Gli attesi sono qualitativi e vanno tarati sul dataset e sui risultati reali.

- **Scenario A (Centralizzato - baseline)**
  - **Contesto**: training centralizzato senza DP/HE.
  - **Assunti**: massima accuracy rispetto ad altri scenari; tempi minori rispetto a FL.
  - **Risultati attesi**: `UseDP=False`, `UseHE=False`, accuracy di riferimento per `Acc_vs_A=0`.

- **Scenario B (FL senza privacy)**
  - **Contesto**: Federated Learning standard (FedAvg), senza DP/HE.
  - **Assunti**: accuracy inferiore a A ma ragionevole; latenza per round > 0.
  - **Risultati attesi**: `UseDP=False`, `UseHE=False`, `HE_MB_per_round` nullo.

- **Scenario C (FL + HE stimato)**
  - **Contesto**: FL con stima overhead HE per comunicazione cifrata.
  - **Assunti**: accuracy simile a B; `HE_MB_per_round > 0` e `HE_MB_total` > 0.
  - **Risultati attesi**: `UseHE=True`, `UseDP=False`.

- **Scenario D (FL + DP)**
  - **Contesto**: FL con rumore DP e clipping; no HE.
  - **Assunti**: accuracy più bassa di B; `Epsilon` e `Delta` presenti.
  - **Risultati attesi**: `UseDP=True`, `UseHE=False`, `HE_MB_per_round` nullo.

- **Scenario E (FL + DP + HE stimato)**
  - **Contesto**: FL con DP e stima HE per la comunicazione.
  - **Assunti**: accuracy ≤ D (o simile); `HE_MB_per_round > 0`.
  - **Risultati attesi**: `UseDP=True`, `UseHE=True`.


In [ ]:
# Controlli eseguibili per ciascuno scenario

def scenario_expectations():
    """Definizione sintetica delle attese per gli scenari A–E.
    Regola i threshold in base ai risultati correnti se necessario.
    """
    return {
        "A": {"expect_use_dp": False, "expect_use_he": False, "min_accuracy": 0.70},
        "B": {"expect_use_dp": False, "expect_use_he": False, "min_accuracy": 0.45},
        "C": {"expect_use_dp": False, "expect_use_he": True,  "min_accuracy": 0.45, "max_he_mb_per_round": 20.0},
        "D": {"expect_use_dp": True,  "expect_use_he": False, "expect_epsilon_not_null": True, "max_accuracy": 0.80},
        "E": {"expect_use_dp": True,  "expect_use_he": True,  "expect_epsilon_not_null": True, "max_he_mb_per_round": 20.0}
    }


def run_all_checks(df: pd.DataFrame):
    exp = scenario_expectations()
    results = {}
    for sc in ["A", "B", "C", "D", "E"]:
        try:
            results[sc] = check_scenario(df, sc, exp.get(sc, {}))
        except Exception as e:
            results[sc] = {"scenario": sc, "passed": False, "violations": [str(e)]}
    return results

if extended_summary_df is not None:
    checks = run_all_checks(extended_summary_df)
    for sc, out in checks.items():
        status = "OK" if out["passed"] else "FAIL"
        print(f"Scenario {sc}: {status} -> {out['violations']}")
else:
    print("[INFO] Nessun summary caricato: salta controlli.")



### Modalità HE: stima vs cifratura reale

Ora la pipeline supporta due modalità:
- `HE_Mode = estimate`: calcola solo la dimensione attesa dei ciphertext senza cifrare.
- `HE_Mode = encrypt`: cifra realmente, somma omomorficamente e decifra l'aggregato.

Nel summary vedremo tempi medi aggiuntivi: `HE_EncTimeMean_s`, `HE_AggTimeMean_s`, `HE_DecTimeMean_s`.


# SNN-IDS Audit Notebook (CSE-CIC-IDS2018)

Questo notebook è pensato per essere auditabile: ogni scelta è documentata file-per-file e riga-per-riga dove rilevante. Include:
- Setup ambiente e dati
- Pipeline dati riproducibile
- Training recipe ottimizzata per tabulari (GRU per finestre temporali)
- Metriche e calibrazione
- Smoke test e test completo (finestre: 5s, 1m, 5m; LR grid)

Note: il codice vive nel repository; il notebook chiama le funzioni senza duplicazioni di logica.


## 1) Setup ambiente

Requisiti minimi:
- Python 3.10+
- pacchetti: pandas, numpy, scikit-learn, tensorflow, matplotlib, seaborn, tqdm

In Colab eseguire le celle seguenti; in locale assicurarsi che `pip install -r requirements.txt` sia stato eseguito.


In [ ]:
# Se sei in Colab, decommenta le righe seguenti
#!git clone --single-branch --branch feat/hyperband-search https://github.com/devedale/snn-ids.git
!git clone https://github.com/devedale/snn-ids.git
#!git clone https://github.com/devedale/snn-ids.git
%cd snn-ids
!pip install -r requirements.txt


Cloning into 'snn-ids'...
remote: Enumerating objects: 921, done.
remote: Counting objects: 100% (570/570), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 921 (delta 258), reused 520 (delta 219), pack-reused 351 (from 2)
Receiving objects: 100% (921/921), 62.45 MiB | 28.91 MiB/s, done.
Resolving deltas: 100% (431/431), done.
/content/snn-ids
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


## 2) Setup dati
Scarica i CSV CSE-CIC-IDS2018 nelle cartelle già attese da `config.py` (attualmente configurato sul path`/content/snn-ids/data`).


In [ ]:
# Importa librerie del progetto
import os, json
import numpy as np
import pandas as pd
from config import DATA_CONFIG, PREPROCESSING_CONFIG, TRAINING_CONFIG, BENCHMARK_CONFIG
from preprocessing.process import preprocess_pipeline
from training.train import train_model
from evaluation.metrics import evaluate_model_comprehensive

print("Config dataset:", DATA_CONFIG["dataset_path"])


In [ ]:
#Uncomment to download full starting dataset
#!curl -L -o ./data/cicids2018.zip  https://www.kaggle.com/api/v1/datasets/download/edoardodalesio/intrusion-detection-evaluation-dataset-cic-ids2018
#!unzip -o ./data/cicids2018.zip  "Tuesday-20-02-2018.csv"   "Wednesday-21-02-2018.csv"  "Thursday-22-02-2018.csv" "Friday-23-02-2018.csv" -d ./data # or #!unzip -o ./data/cicids2018.zip -d ./data



#Uncomment to use preprocessed cache
!mkdir preprocessed_cache
!curl -L -o ./preprocessed_cache/preprocessed_cache.zip  https://www.kaggle.com/api/v1/datasets/download/edoardodalesio/cic-ids-2018-benign-vs-attack
!unzip -o ./preprocessed_cache/preprocessed_cache.zip -d ./preprocessed_cache
!touch data/Friday-02-03-2018.csv data/Friday-16-02-2018.csv data/Friday-23-02-2018.csv data/Thursday-01-03-2018.csv data/Thursday-15-02-2018.csv data/Thursday-22-02-2018.csv data/Tuesday-20-02-2018.csv data/Wednesday-14-02-2018.csv data/Wednesday-21-02-2018.csv data/Wednesday-28-02-2018.csv
#



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  9.8G  100  9.8G    0     0  73.7M      0  0:02:17  0:02:17 --:--:-- 71.6M
Archive:  ./preprocessed_cache/preprocessed_cache.zip
  inflating: ./preprocessed_cache/Friday-02-03-2018/attack_records.csv  
  inflating: ./preprocessed_cache/Friday-02-03-2018/benign_records.csv  
  inflating: ./preprocessed_cache/Friday-16-02-2018/attack_records.csv  
  inflating: ./preprocessed_cache/Friday-16-02-2018/benign_records.csv  
  inflating: ./preprocessed_cache/Friday-23-02-2018/attack_records.csv  
  inflating: ./preprocessed_cache/Friday-23-02-2018/benign_records.csv  
  inflating: ./preprocessed_cache/Thursday-01-03-2018/attack_records.csv  
  inflating: ./preprocessed_cache/Thursday-01-03-2018/benign_records.csv  
  inflating: ./preprocessed_cache/Thursday

In [ ]:
!python3 benchmark.py --hyperband --model mlp_4_layer --hb-max-epochs 20 --hb-final-epochs 30 --hb-batch-size 128

Output streaming troncato alle ultime 5000 righe.
48/48 ━━━━━━━━━━━━━━━━━━━━ 18s 319ms/step - accuracy: 0.0218 - loss: 3872229.2500 - val_accuracy: 0.0291 - val_loss: 684934.6875
Epoch 2/2
28/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0352 - loss: 579393.8125     
Epoch 2: Calculating per-class training loss...
  - Class 0: Loss = 308888.7812
  - Class 1: Loss = 223672.5469
  - Class 2: Loss = 106.8611
  - Class 3: Loss = 121006.7891
  - Class 4: Loss = 374988.5000
  - Class 5: Loss = 0.0000
  - Class 6: Loss = 863203.1875
  - Class 7: Loss = 159326.4531
  - Class 8: Loss = 333213.3750
  - Class 9: Loss = 13741.2773
  - Class 10: Loss = 20443.7324
  - Class 11: Loss = 1392942.6250
  - Class 12: Loss = 794374.1250
  - Class 13: Loss = 108316.2969
  - Class 14: Loss = 163327.1250
  - Class 15: Loss = 1022416.5625
  - Class 16: Loss = 834491.8750
  - Class 17: Loss = 111324.5703
  - Class 18: Loss = 184165.8438
  - Class 19: Loss = 272886.9688
  - Class 20: Loss = 257115.8906
  - C

In [ ]:
!zip "mlp_4_layer_results_$(date +%Y%m%d_%H%M%S).zip" *.zip
from google.colab import drive
drive.mount('/content/drive')

# Copia lo zip in Drive
!cp mlp_4_layer_results_* /content/drive/MyDrive/

  adding: mlp_analysis_results_20250826_220126/ (stored 0%)


Esempi di utilizzo:

  # 1. Eseguire un test rapido (smoke test) per verificare che tutto funzioni
  python3 benchmark.py --smoke-test

  # 2. Eseguire un singolo test con un modello specifico e dimensione del campione
  python3 benchmark.py --model gru --sample-size 20000

  # 3. Eseguire il benchmark completo su tutti i modelli e iperparametri di default
  python3 benchmark.py --full

  # 4. Eseguire il benchmark completo con una dimensione del campione personalizzata
  python3 benchmark.py --full --sample-size 50000

  # 5. Eseguire un singolo test specificando iperparametri custom (nota: devono essere nel formato atteso dal modulo di training)
  python3 benchmark.py --model lstm --epochs 15 --batch-size 128 --learning-rate 0.0005
        '''
    )
    
    # Argomenti principali per la selezione della modalità
    parser.add_argument('--smoke-test', action='store_true', help='Esegue uno smoke test veloce e leggero.')
    parser.add_argument('--full', action='store_true', help='Esegue il benchmark completo su più modelli e iperparametri.')
    
    # Argomenti per la configurazione di base
    parser.add_argument('--sample-size', type=int, help='Numero totale di campioni da utilizzare (BENIGN + ATTACK).')
    parser.add_argument('--data-path', type=str, help='Path alla directory contenente i file CSV del dataset.')
    parser.add_argument('--output-dir', type=str, default='benchmark_results', help='Directory per salvare i risultati.')

    # Argomenti per la configurazione del modello (usati in test singoli o come override)
    parser.add_argument('--model', choices=['dense', 'gru', 'lstm'], help='Tipo di modello da testare in un singolo run.')
    parser.add_argument('--epochs', type=int, help="Override del numero di epoche per il training (es. 10).")
    parser.add_argument('--batch-size', type=int, help="Override della batch size per il training (es. 64).")
    parser.add_argument('--learning-rate', type=float, help="Override del learning rate (es. 0.001).")
    
    args = parser.parse_args()

## 3) Smoke test (GRU)
Esegue pipeline ridotta per verificare fine-to-end: bilanciamento security, IP→ottetti, finestre, training GRU (K-Fold), valutazione con PNG.


In [ ]:
# Smoke test
from sklearn.model_selection import train_test_split

# Override per test rapido
PREPROCESSING_CONFIG['sample_size'] = 3000
TRAINING_CONFIG['model_type'] = 'gru'
TRAINING_CONFIG['hyperparameters']['epochs'] = [2]
TRAINING_CONFIG['hyperparameters']['batch_size'] = [32]

X, y, label_encoder = preprocess_pipeline()
model, log, model_path = train_model(X, y, model_type='gru')

# Valutazione rapida
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
report = evaluate_model_comprehensive(model, X_te, y_te, class_names=label_encoder.classes_.tolist(), output_dir='notebook_eval/smoke')
report['basic_metrics']['accuracy']


## 4) Test completo (GRU) con finestre 5s, 1m, 5m e grid LR
In questo test variamo:
- finestre temporali: `window_size` e `step` coerenti con risoluzioni 5s, 1m, 5m
- learning rate: `[1e-3, 5e-4, 1e-4]`
- epoche moderate per tempi ragionevoli


In [ ]:
from copy import deepcopy

results = []
base_prep = deepcopy(PREPROCESSING_CONFIG)
base_train = deepcopy(TRAINING_CONFIG)

# Grid finestre (timesteps) e learning rate
window_configs = [
    {"name": "5s", "window_size": 10, "step": 5},
    {"name": "1m", "window_size": 60//6, "step": 10},  # es: 10 step
    {"name": "5m", "window_size": 50, "step": 10},
]
lr_grid = [1e-3, 5e-4, 1e-4]

for wc in window_configs:
    PREPROCESSING_CONFIG['use_time_windows'] = True
    PREPROCESSING_CONFIG['window_size'] = wc['window_size']
    PREPROCESSING_CONFIG['step'] = wc['step']

    for lr in lr_grid:
        TRAINING_CONFIG['model_type'] = 'gru'
        TRAINING_CONFIG['hyperparameters']['epochs'] = [5]
        TRAINING_CONFIG['hyperparameters']['batch_size'] = [64]
        TRAINING_CONFIG['hyperparameters']['learning_rate'] = [lr]

        print(f"\n=== Config: {wc['name']} | lr={lr} ===")
        X, y, le = preprocess_pipeline()
        model, log, path = train_model(X, y, model_type='gru')
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        rep = evaluate_model_comprehensive(model, X_te, y_te, le.classes_.tolist(), output_dir=f'notebook_eval/{wc["name"]}_lr{lr}')
        results.append({'window': wc['name'], 'lr': lr, 'accuracy': rep['basic_metrics']['accuracy']})

# Ripristina config
PREPROCESSING_CONFIG.update(base_prep)
TRAINING_CONFIG.update(base_train)

pd.DataFrame(results).sort_values('accuracy', ascending=False).head()


## 5) Riproducibilità
Impostiamo i seed per rendere i risultati ripetibili (entro i limiti dell'hardware).


In [ ]:
import os, random
import numpy as np
import tensorflow as tf

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print('Seed impostato:', SEED)


## 6) Audit dati e feature
Controlliamo distribuzione classi, percentuali, e presenza di attacchi rilevanti nel sample selezionato.


In [ ]:
from collections import Counter

def audit_distribution(y, label_encoder):
    counts = Counter(y)
    classes = label_encoder.classes_.tolist()
    dist = {classes[i]: int(counts.get(i, 0)) for i in range(len(classes))}
    total = sum(dist.values())
    df = pd.DataFrame({
        'classe': list(dist.keys()),
        'conteggio': list(dist.values())
    }).sort_values('conteggio', ascending=False)
    df['percentuale'] = (df['conteggio'] / total * 100).round(2)
    return df

# Esempio live (riutilizza X,y,label_encoder se esistono)
try:
    audit_distribution(y, label_encoder)
except Exception as e:
    print('Esegui prima il smoke test per generare X,y,label_encoder')


## 7) Metriche e calibrazione
Oltre alle metriche standard, aggiungiamo ECE (Expected Calibration Error) per valutare la calibrazione delle probabilità.


In [ ]:
import numpy as np

def expected_calibration_error(y_true, y_proba, n_bins=10):
    # binning su max probability
    confidences = y_proba.max(axis=1)
    predictions = y_proba.argmax(axis=1)
    accuracies = (predictions == y_true).astype(float)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        mask = (confidences > bins[i]) & (confidences <= bins[i+1])
        if mask.any():
            avg_conf = confidences[mask].mean()
            avg_acc = accuracies[mask].mean()
            ece += np.abs(avg_acc - avg_conf) * mask.mean()
    return float(ece)

# Esempio: usa il modello dallo smoke test, se disponibile
try:
    y_proba = model.predict(X_te, verbose=0)
    print('ECE:', expected_calibration_error(y_te, y_proba, n_bins=15))
except Exception as e:
    print('Esegui prima smoke test e valutazione per avere y_te e y_proba')


## 8) Documentazione file-per-file
In questa sezione spieghiamo le scelte implementative nei file chiave: `preprocessing/process.py`, `training/train.py`, `evaluation/metrics.py`, `benchmark.py` e `config.py`.


In [ ]:
import inspect, textwrap
import preprocessing.process as P
import training.train as T
import evaluation.metrics as E
import benchmark as B
import config as C

def show_source(obj, start=None, end=None):
    src = inspect.getsource(obj)
    if start or end:
        lines = src.splitlines()
        src = "\n".join(lines[start:end])
    print(textwrap.dedent(src))

print('--- config.py (sezioni principali) ---')
print('DATA_CONFIG:'); print(C.DATA_CONFIG)
print('\nPREPROCESSING_CONFIG:'); print(C.PREPROCESSING_CONFIG)
print('\nTRAINING_CONFIG:'); print(C.TRAINING_CONFIG)

print('\n--- preprocessing.process: load_and_balance_dataset ---')
show_source(P.load_and_balance_dataset)
print('\n--- preprocessing.process: preprocess_pipeline ---')
show_source(P.preprocess_pipeline)

print('\n--- training.train: _train_k_fold ---')
show_source(T._train_k_fold)
print('\n--- training.train: _train_split ---')
show_source(T._train_split)

print('\n--- evaluation.metrics: evaluate_model_comprehensive ---')
show_source(E.evaluate_model_comprehensive)

print('\n--- benchmark.SNNIDSBenchmark (run_smoke_test) ---')
show_source(B.SNNIDSBenchmark.run_smoke_test)


### Note progettuali
- Zero hard-code: tutte le scelte sono in `config.py`; il notebook applica override solo per esperimenti.
- Pipeline riproducibile: sampling e bilanciamento documentati; seed fissati.
- Training recipe tabulari: GRU su finestre 3D, scaling per-fold, StratifiedKFold.
- Metriche e PNG: confusion matrix dettagliata, cybersecurity, ROC, accuracy per classe, ECE.
- Notebook auditabile: usa `inspect` per mostrare il codice sorgente eseguito.


## 9) Limitazioni
- I risultati su classi rare vanno interpretati con cautela; forniamo sempre breakdown per‑classe.
- La calibrazione (ECE) è informativa ma non esaustiva.
- Il bilanciamento “security” riduce bias ma non sostituisce protocolli di acquisizione realistici.
- Evitiamo leakage scalando per‑fold; ulteriori audit sono comunque consigliati in ambienti operativi.
- Per produzione sono necessarie valutazioni cost‑sensitive e monitoraggio del drift.


## Aggiornamento: Seeding centralizzato e riproducibilità
Questa sezione integra il notebook mantenendo intatti i blocchi precedenti.

- Il seed unico del progetto è definito in `config.py` come `RANDOM_CONFIG['seed']` (default: 79).
- Per massimizzare la riproducibilità, usiamo `set_global_seed(SEED)` che imposta i seed per Python, NumPy e TensorFlow.
- Per `random_state` in Scikit‑Learn (es. `KFold`, `StratifiedKFold`, `train_test_split`), usare sempre il seed da config.

Le celle successive impostano il seed centralizzato da usare in tutto il notebook.


In [ ]:
# Importa config e utility di seeding dal repository
import os, sys
from pathlib import Path

# Aggiunge il project root al sys.path se serve
ROOT = Path.cwd()
if (ROOT / 'config.py').exists():
    sys.path.append(str(ROOT))

from config import RANDOM_CONFIG
from src.utils import set_global_seed

SEED = int(RANDOM_CONFIG.get('seed', 79))
set_global_seed(SEED)

# Mantiene a disposizione una variabile RS da usare come random_state Sklearn
RS = SEED
RS
